#注意:
此文件已在colab上验证

#配置colab环境

In [1]:
#colab中运行jupyter文件的步骤：
# 1.挂载云盘
from google.colab import drive
drive.mount('/content/gdrive')

# 2.安装需要的软件
!pip3 install transformers
!pip3 install pytorch-crf

import os
def get_root_dir():
    if os.path.exists('/content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/grade'):
        return '/content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/grade/'
    else:
        return './' #在本地

# 3.调用系统命令，切换到对应工程路径，相当于cd，但是直接!cd是不行的
print("path:",get_root_dir())
os.chdir(get_root_dir())

# 4.再次确认路径
!pwd
!ls

Mounted at /content/gdrive
     |████████████████████████████████| 3.4 MB 14.8 MB/s 
     |████████████████████████████████| 895 kB 66.4 MB/s 
     |████████████████████████████████| 61 kB 679 kB/s 
     |████████████████████████████████| 3.3 MB 61.9 MB/s 
     |████████████████████████████████| 596 kB 81.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
path: /content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/grade/
/content/gdrive/MyDrive/第二次完成面试机器人_toColab/src/grade
grade.h5       意图识别推理及评测.ipynb		  打分算法-基于文本分类.ipynb
parameter.pkl  意图识别（文本分类）-最新版.ipynb  打分算法-推理.ipynb


In [7]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器
import torch
import pickle as pk
import numpy as np
import jieba
import pdb

class Grade(nn.Module):
    def __init__(self, parameter):
        super(Grade, self).__init__()
        embedding_dim = parameter['embedding_dim']
        hidden_size = parameter['hidden_size']
        num_layers = parameter['num_layers']
        dropout = parameter['dropout']
        word_size = parameter['word_size']
        self.embedding = nn.Embedding(word_size, embedding_dim, padding_idx=0)
        
        self.lstm_q = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)

        self.lstm_a = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)

        self.dropout = nn.Dropout(p=dropout)

        
    def forward(self, q, a1,a2 = None):
        q_emd = self.embedding(q)
        q_emd,(h, c)= self.lstm_q(q_emd)
        q_emd = torch.max(q_emd,1)[0]

        a1_emd = self.embedding(a1)
        a1_emd,(h, c)= self.lstm_a(a1_emd)
        a1_emd = torch.max(a1_emd,1)[0]
        if a2 is not None:
            a2_emd = self.embedding(a2)
            a2_emd,(h, c)= self.lstm_a(a2_emd)
            a2_emd = torch.max(a2_emd,1)[0]
            return q_emd,a1_emd,a2_emd
        return F.cosine_similarity(q_emd,a1_emd,1,1e-8)


    
def list2torch(a):
    return torch.from_numpy(np.array(a)).long().to(parameter['cuda'])

def predict(model,parameter,q,a):
    #pdb.set_trace()
    q = list(q)
    a = list(a)
    q_cut = []
    for i in q:
        if i in parameter['word2id']:
            q_cut.append(parameter['word2id'][i])
        else:
            q_cut.append(parameter['word2id']['<UNK>'])
    a_cut = []
    for i in a:
        if i in parameter['word2id']:
            a_cut.append(parameter['word2id'][i])
        else:
            a_cut.append(parameter['word2id']['<UNK>'])
    print(q_cut,a_cut)
    q_cut,a_cut = [q_cut[:parameter['max_len']]],[a_cut[:parameter['max_len']]]
    prob = model(list2torch(q_cut),list2torch(a_cut))
    print(prob)
    return prob.cpu().item()

def load_model(root_path = './'):
    parameter = pk.load(open(root_path+'parameter.pkl','rb'))
    model = Grade(parameter).to(parameter['cuda'])
    model.load_state_dict(torch.load(root_path+'grade.h5'))
    model.eval()
    return model,parameter

In [8]:
model,parameter = load_model()
q = '特征工程选择思路？'
a = '基于统计信息的，熵、相关性、KL系数'
prob = predict(model,parameter,q,a)
prob

[13, 14, 21, 22, 15, 16, 23, 24, 25] [30, 31, 50, 51, 52, 53, 29, 32, 54, 55, 56, 57, 58, 55, 59, 7, 35, 36]
tensor([0.6987], device='cuda:0', grad_fn=<DivBackward0>)


0.6986643671989441